In [ ]:
# Transformers installation
# ! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Zero-shot image classification

Zero-shot image classification is a task that involves classifying images into different categories using a model that was
not explicitly trained on data containing labeled examples from those specific categories.

Traditionally, image classification requires training a model on a specific set of labeled images, and this model learns to
"map" certain image features to labels. When there's a need to use such model for a classification task that introduces a
new set of labels, fine-tuning is required to "recalibrate" the model.

In contrast, zero-shot or open vocabulary image classification models are typically multi-modal models that have been trained on a large
dataset of images and associated descriptions. These models learn aligned vision-language representations that can be used for many downstream tasks including zero-shot image classification.

This is a more flexible approach to image classification that allows models to generalize to new and unseen categories
without the need for additional training data and enables users to query images with free-form text descriptions of their target objects .

In this guide you'll learn how to:

* create a zero-shot image classification pipeline
* run zero-shot image classification inference by hand

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install -q transformers
```

In [ ]:
!pip install -q transformers

In [ ]:
from PIL import Image
import requests
import os

## Zero-shot image classification pipeline

The simplest way to try out inference with a model supporting zero-shot image classification is to use the corresponding [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline).
Instantiate a pipeline from a [checkpoint on the Hugging Face Hub](https://huggingface.co/models?pipeline_tag=zero-shot-image-classification&sort=downloads):

In [ ]:
from transformers import pipeline

checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")

Next, choose an image you'd like to classify.

In [ ]:
# Specify the folder path
folder_path = "images_pca/"

# Set the file name
file_name = "1.jpg"

# Combine the folder path and file name to create the full path
local_image_path = os.path.join(folder_path, file_name)

# Open the local image file
image = Image.open(local_image_path)
print(image)

In [ ]:
image

In [ ]:
# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only image files (assuming common image file extensions)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]


In [ ]:
interior_labels =["living room", "bedroom","kitchen", "dinning room","bathroom", "GYM Room","hallway","stairway"]
exterior_labels = [ "front yard","backyard", "patio", "deck","pool","drive lane","driveway","parking lot"]
labels = interior_labels + exterior_labels
print(labels)
clip_labels = [f"{label} inside a house" for label in interior_labels] + [f"{label} outside" for label in exterior_labels] 
            
print(clip_labels)

In [ ]:
predictions = detector(image, candidate_labels=labels)
predictions

In [ ]:
predictions = detector(image, candidate_labels=clip_labels)
predictions

In [ ]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

model = AutoModelForZeroShotImageClassification.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

In [ ]:
prompts = [
    "A living room in excellent condition, very clean and well-maintained.",
    "A living room in good condition, mostly clean with minor wear.",
    "A living room in average condition, visibly used but not dirty.",
    "A living room in poor condition, quite messy and needs cleaning.",
    "A living room in very poor condition, extremely dirty and neglected."
]

# Preprocess and encode the text prompts
text_tokens = clip.tokenize(prompts).to(device)

In [ ]:
# encode a image with labels
inputs = processor(images=image, text=clip_labels, return_tensors="pt", padding=True)

import torch

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

result = [
    {"score": score, "label": clip_label}
    for score, clip_label in sorted(zip(probs,clip_labels), key=lambda x: -x[0])
]

result

In [ ]:
images = []
for file_name in image_files:
    # Combine the folder path and file name to create the full path
    local_image_path = os.path.join(folder_path, file_name)
    image = Image.open(local_image_path)
    images.append(image)

results = []
    
for image in images:
    inputs = processor(images=image, text=clip_labels, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits_per_image[0]
    probs = logits.softmax(dim=-1).numpy()
    scores = probs.tolist()

    result = [
        {"score": score, "label": clip_label}
        for score, clip_label in sorted(zip(probs, clip_labels), key=lambda x: -x[0])
    ]

    results.append(result)



In [ ]:
# Output results
for idx, result in enumerate(results):
    print(f"Results for image {idx + 1}:")
    images[idx].show()
    for res in result[:3]:
        print(res)

In [ ]:
results_det= []
for image in images:
    predictions = detector(image, candidate_labels=clip_labels)
    results_det.append(predictions)

In [ ]:
for idx, result in enumerate(results_det):
    print(f"Results for image {idx + 1}:")
    images[idx].show()
    for res in result[:3]:
        print(res)

In [ ]:
##condition labels

In [ ]:
condition_labels = ["excellent condition","good condition", "average condition","bad condition","unacceptable condition"]

condition_labels = [f"the property is in {label}" for label,desc_label in zip(condition_labels,desc_labels) ] 

# condition_labels = ["best","better", "good","worse","worst"]
# condition_labels = [f"physical condition evaluation is {label}" for label in condition_labels ] 
condition_labels

In [ ]:
results_cond=[]

for image in images:
    inputs = processor(images=image, text=condition_labels, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits_per_image[0]
    probs = logits.softmax(dim=-1).numpy()
    scores = probs.tolist()

    result = [
        {"score": score, "label": clip_label}
        for score, clip_label in sorted(zip(probs, condition_labels), key=lambda x: -x[0])
    ]

    results_cond.append(result)

In [ ]:
results_cond=[]
for image in images:
    predictions = detector(image, candidate_labels=condition_labels )
    results_cond.append(predictions)

In [ ]:
for idx, result in enumerate(results_cond):
    print(f"Results for image {idx + 1}:")
    images[idx].show()
    for res in result[:3]:
        print(res)

In [ ]:
predictions = detector(image, candidate_labels=condition_labels )

## Zero-shot image classification by hand

Now that you've seen how to use the zero-shot image classification pipeline, let's take a look how you can run zero-shot
image classification manually.

Start by loading the model and associated processor from a [checkpoint on the Hugging Face Hub](https://huggingface.co/models?pipeline_tag=zero-shot-image-classification&sort=downloads).
Here we'll use the same checkpoint as before:

In [ ]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

model = AutoModelForZeroShotImageClassification.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

Let's take a different image to switch things up.